# Исследование надежности заемщиков.

### Шаг 1. Обзор данных

In [3]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [4]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

**Выведем количество пропущенных значений для каждого столбца. Используем комбинацию двух методов.**

In [6]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце мы обработаеем на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполним пропуски в этом столбце медианным значением по каждому типу из столбца `income_type`.**

In [7]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

**В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода `abs()`.**

In [8]:
data['days_employed'] = data['days_employed'].abs()

**Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.**

In [9]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть.

**Выведем перечень уникальных значений столбца `children`.**

In [10]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

**В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [11]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [12]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

**Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [13]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**Убедимся, что все пропуски заполнены. Провериим себя и ещё раз выведем количество пропущенных значений для каждого столбца с помощью двух методов.**

In [14]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

**Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [15]:
data['total_income'] = data['total_income'].astype(int)

**Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру.**

In [16]:
data['education'] = data['education'].str.lower()

**Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.**

In [17]:
data.duplicated().sum()

71

In [18]:
data = data.drop_duplicates()

### Категоризация данных

**Задание 16. На основании диапазонов, указанных ниже, создайте в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используйте собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [19]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [20]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Задание 17. Выведите на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [21]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Задание 18. Создайте функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используйте собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучите данные в столбце `purpose` и определите, какие подстроки помогут вам правильно определить категорию.**

In [22]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [23]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследуйте данные и ответьте на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [29]:
debt_children = pd.DataFrame()
#посчитаем количество значений наличия задолженности по возврату кредитов по категории children 
debt_children['count_children'] = data.groupby('children')['debt'].count() 
#посчитаем сумму задолженности по возврату кредитов по категории children 
debt_children['sum_children'] = data.groupby('children')['debt'].sum()
#посчитаем процетное соотношение
debt_children['result_children'] = round(debt_children['sum_children'] / debt_children['count_children'] * 100, 2) 
debt_children


,count_children,sum_children,result_children
children,,,
0,14091,1063,7.54
1,4808,444,9.23
2,2052,194,9.45
3,330,27,8.18
4,41,4,9.76
5,9,0,0.00


**Вывод:** Из сформированной таблицы можно сделать вывод, что наиболее высокий процент задолжности (9,76) у семей, имеющих 4 детей, но при этом задолженность семей без детей ниже только на 2,22 %.  

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [28]:
debt_family_status = pd.DataFrame()
#посчитаем количество значений наличия задолженности по возврату кредитов по категории family_status 
debt_family_status['family_status_count'] = data.groupby('family_status')['debt'].count()
 #посчитаем сумму задолженности по возврату кредитов по категории family_status 
debt_family_status['family_status_sum'] = data.groupby('family_status')['debt'].sum()
#посчитаем процетное соотношение
debt_family_status['result_family_status'] = round(debt_family_status['family_status_sum'] /\
                                                   debt_family_status['family_status_count'] * 100, 2) 
debt_family_status

,family_status_count,family_status_sum,result_family_status
family_status,,,
Не женат / не замужем,2796,273,9.76
в разводе,1189,84,7.06
вдовец / вдова,951,63,6.62
гражданский брак,4134,385,9.31
женат / замужем,12261,927,7.56


**Вывод:** Глядя на таблицу, можно заметить, люди, находящиеся в разводе и вдовы/вдовцы исправнее выплачивают кредиты, чем люди не в браке и не бывавшие в браке.

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [26]:
debt_total_income = pd.DataFrame()
debt_total_income['sum'] = data.groupby('total_income_category')['debt'].sum()
debt_total_income['count'] = data.groupby('total_income_category')['debt'].count()
debt_total_income['ratio'] = round(debt_total_income['sum'] / debt_total_income['count'] * 100, 2)
debt_total_income.sort_values('total_income_category', ascending = True)



,sum,count,ratio
total_income_category,,,
A,2,25,8.00
B,354,5014,7.06
C,1353,15921,8.50
D,21,349,6.02
E,2,22,9.09


**Вывод:** Самая высокая возвратность у людей с доходом от 30 до 50 тысяч.Люди с доходом от 500 до 1000 тысяч также имеют низкую долю просрочки по кредиту. Люди как с доходом до 300000, так и с доходом от 50 до 200 имеют высокую вероятность, относительно остальных, не вернуть кредит.Возможно это связано с тем, что эти люди имеют детей и еще несколько кредитов, поэтому вероятность невозврата кредита у таких людей выше.

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [30]:
debt_purpose_category = pd.DataFrame()
#посчитаем количество значений наличия задолженности по возврату кредитов по категории purpose_category 
debt_purpose_category['purpose_category_count'] = data.groupby('purpose_category')['debt'].count()
#посчитаем сумму задолженности по возврату кредитов по категории purpose_category 
debt_purpose_category['purpose_category_sum'] = data.groupby('purpose_category')['debt'].sum() 
 #посчитаем процетное соотношение
debt_purpose_category['result_purpose'] = \
round(debt_purpose_category['purpose_category_sum'] / debt_purpose_category['purpose_category_count'] * 100, 2)
debt_purpose_category.sort_values('result_purpose', ascending = True)

,purpose_category_count,purpose_category_sum,result_purpose
purpose_category,,,
операции с недвижимостью,10751,780,7.26
проведение свадьбы,2313,183,7.91
получение образования,3988,369,9.25
операции с автомобилем,4279,400,9.35


**Вывод:** Из 4 целевых категорий, кредиты чаще всего платят в срок по операциям с недвижимостью и по проведению свадьбы, разброс между максимальным значением примерно 2%. 

#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

*Ответ:*Пропуски присутствовали в столбцах days_employed и total_income. Чаще всего причиной пропусков является человеческий фактор (либо по ошибке не заполнены, либо не захотели заполнять. Также по двум столбацам количество пропусков одинаковое.

#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* Потому что медиана в отличие от среднего арифметического игнорирует выбросы данных.

### Шаг 4: общий вывод.

**Цель исследования:** необходимо выявить влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

В ходе выполнения исследования выявлено, что в двух столбцах days_employed (общий трудовой стаж в днях) и total_income (ежемесячный доход) были пропущенны значения, total_income — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому пропуски заполнены в этом столбце медианным значением по каждому типу из столбца income_type (тип занятости).

Проведена обработка аномальных значений, обработано отрицательное значение количества дней трудового стажа в столбце days_employed и два аномальных значения в столбце children.

В столбце total_income тип данных вещественный заменен на целочисленный.Также в столбце education обработаны неявные дубликаты, так как имелись одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. 

Разберем подробнее каждый фактор влияния на погашение кредита:

1) Влияние наличия детей на вероятность просрочки платежа (по количеству детей в семье)

Нет детей - 7.54%

1 ребенок - 9.23%

2 ребенка - 9.45%

4 детей - 9.76%

Абсолютная разница между минимальным и максимальным значением - 2.22%.

Относительная разница между минимальным и максимальным значением - 22.7%.

Из этого можно сделать вывод, что чем больше детей у заемщика, тем больше вероятность просрочки. 

Семьи, у которых 5 детей, и семьи, у которых нет детей, либо не допускают вовсе, либо допускают незначительно просрочку кредита. Это может быть связано с тем, семьи, у которых 5 детей более отвественные, а семьи, которых нет детей более платежеспособны. Люди, имеющие 1 ребенка, допускают просрочку на 2.3% меньше, чем семьи, имеющие двух детей, что подтверждает сделанный вывод.

2) Влияние семейного положения на вероятность просрочки платежа

не женат / не замужем - 9.76%

в разводе - 7.06%

вдовец / вдова - 6.62%

гражданский брак - 9.31%

женат / замужем - 7.56%

Абсолютная разница между минимальным и максимальным значением - 3.14%

Относительная разница между минимальным и максимальным значением - 32.17%

Из этого можно сделать вывод, что люди, находящиеся в разводе и вдовы/вдовцы исправнее выплачивают кредиты, чем люди не в браке и не бывавшие в браке. Объяснить это можно также тем, что первые более платежеспособные, чем семейные люди, либо ответственнее.

3) Влияние цели кредита на вероятность возврата кредита в срок 

операции с недвижемостью - 7.26%

проведение свадьбы - 7.91%

получение образования - 9.25%

операции с автомобилями - 9.35%

Абсолютная разница между минимальным и максимальным значением - 2.09%.

Относительная разница между минимальным и максимальным значением - 22.35%.

Из 4 целевых категорий, кредиты чаще всего платят в срок по операциям с недвижимостью и по проведению свадьбы.

4) Влияние уровня дохода на просрочку платежа

0 - 30000 - 9.09%

От 30001 до 50000 - 6.02%

От 50001 до 200000- 8.50%

От 200001 до 1000000 - 7.06%

От 1000000 и выше - 8.00%

Абсолютная разница между минимальным и максимальным значением - 3.07%.

Относительная разница между минимальным и максимальным значением - 33.77%.

*Вывод:* самая высокая возвратность у людей с доходом от 30 до 50 тысяч.Люди с доходом от 500 до 1000 тысяч также имеют низкую долю просрочки по кредиту. Люди как с доходом до 300000, так и с доходом от 50 до 200 имеют высокую вероятность, относительно остальных, не вернуть кредит.Возможно это связано с тем, что эти люди имеют детей и еще несколько кредитов, поэтому вероятность невозврата кредита у таких людей выше.

Из проделанной работы можно сделать вывод, что наиболее платежеспособными являются разведенные, вдовцы/вдовы, либо неженатые люди с доходом выше среднего и неимеющие детей. Наименьшее влияние на погашение кредита оказывают его цели.